In [24]:
import pandas as pd
import numpy as np
import utils
import Dotblot
import GeneralDilution

from importlib import reload
import os
import re
import json

In [74]:
reload(utils)
reload(Dotblot)
dotblot_method = Dotblot.DotblotMethod()

data = utils.import_excel_dotblot(r"L:/Departements/BTDS_AD/002_AFFS/Lab Automation/09. Tecan/06. DotBlot_automation_DPP/DotBlot_automation_dilution_template_final.xlsx")
sample_dilution_data = data[0]
coating_protein_dilution_data = data[1]
pos_control_dilution_data = data[2]
neg_control_dilution_data = data[3]

In [77]:

# Reset parameters
dotblot_method.last_eppendorf_pos = 1
dotblot_method.last_deep_well_pos = 1

dotblot_method.n_pos_control_steps = len(pos_control_dilution_data["Assay buffer volume"])
dotblot_method.n_neg_control_steps = len(neg_control_dilution_data["Assay buffer volume"])
dotblot_method.sample_dilution_data = {col: sample_dilution_data[col].values.astype(float).tolist() for col in sample_dilution_data.columns}
dotblot_method.pos_control_dilution_data = {col: pos_control_dilution_data[col].values.astype(float).tolist() for col in pos_control_dilution_data.columns}
dotblot_method.neg_control_dilution_data = {col: neg_control_dilution_data[col].values.astype(float).tolist() for col in neg_control_dilution_data.columns}
dotblot_method.pos_control_vial_posX = 1
dotblot_method.pos_control_vial_posY = 1
dotblot_method.main_sample_labware_type = "Eppendorf"
dotblot_method.main_sample_dil_destination = "DeepWell"
dotblot_method.n_samples_main_dilution = 4
dotblot_method.n_sample_dilution_steps = len(sample_dilution_data["Assay buffer volume"])

# read JSON assay file and generate methods and products lists
RAW_ASSAYS_DATA, METHODS_LIST, PRODUCTS_DICT = utils.generate_methods_and_products("assays.json")
indices = utils.get_assay_indices(RAW_ASSAYS_DATA, "Dotblot", "CD79bxCD20xCD3")
dotblot_method.pump_steps_data = RAW_ASSAYS_DATA[indices[0]]["step_types"] # get pump steps from JSON file

dotblot_method.dotblot()

pos dilutions done
negative ctr eppendorf pos: [2]
neg dilutions done
las eppendorf pos:  3
sample last eppensorf pos:  7
sample dilutions done
sample eppendorf pos: [7, 8, 9, 10]
calculated pump labware positions
transfer volume instruction  1 done
transfer volume instruction  2 done
transfer volume instruction  3 done
transfer volume instruction  4 done
transfer volume instruction  5 done
transfer volume instruction  6 done
transfer volume instruction  7 done
transfer volume instruction  8 done
transfer volume instruction  9 done
generated all GWL files


([1], [2], [7, 8, 9, 10])

In [72]:
sample_dilution_data

,Dilution step,Initial concentration,Withdrawn volume,Final concentration,Assay buffer volume,Sample volume,Final Volume
0,1,10.2,6,0.08,759.0,6,765.0
1,2,0.08,10,0.002,390,10,400
2,3,0.002,100,0.0002,900,100,1000


In [20]:
csv_path = r'L:\Departements\BTDS_AD\002_AFFS\Lab Automation\09. Tecan\01. Methods\3. DotBlot\2. Positive control.csv'

mix_data = []
labware_name = "Custom holder[001]"
labware_pos = 1
volume = 300

for i in range(3):
    mix_data.append([labware_name, labware_pos, labware_name, labware_pos, volume])

mix_data
mix_csv = pd.DataFrame(mix_data, columns=None)
mix_csv

original_csv = pd.read_csv(csv_path, header=None) # create dataframe from original CSV file
final_csv = pd.concat([original_csv, mix_csv], ignore_index=True) # concatenate both dataframes into one
final_csv.to_csv(csv_path, index=False, header=False)

In [ ]:
pump_step_data_simple = [{'step_type': 'Transfer volume to wells', 'volume_amount': '100', 'liquid_type': 'DPBS', 'wells_type': 'All wells'}, {'step_type': 'Vacuum'}, {'step_type': 'Wait timer', 'wait_timer': 10.0}]
pump_step_data = [{'step_type': 'Transfer volume to wells', 'volume_amount': '100', 'liquid_type': 'Samples', 'wells_type': 'All wells'}, {'step_type': 'Vacuum'}, {'step_type': 'Transfer volume to wells', 'volume_amount': '100', 'liquid_type': 'Coating protein', 'wells_type': 'All wells'}, {'step_type': 'Wait timer', 'wait_timer': 10.0}, {'step_type': 'Vacuum'}, {'step_type': 'Transfer volume to wells', 'volume_amount': '200', 'liquid_type': 'Blocking buffer', 'wells_type': 'All wells'}, {'step_type': 'Wait timer', 'wait_timer': 10.0}, {'step_type': 'Vacuum'}, {'step_type': 'Transfer volume to wells', 'volume_amount': '100', 'liquid_type': 'Pos/Neg control', 'wells_type': 'Only samples'}, {'step_type': 'Vacuum'}, {'step_type': 'Transfer volume to wells', 'volume_amount': '100', 'liquid_type': 'Conjugate', 'wells_type': 'All wells'}, {'step_type': 'Wait timer', 'wait_timer': 10.0}, {'step_type': 'Vacuum'}, {'step_type': 'Transfer volume to wells', 'volume_amount': '200', 'liquid_type': 'DPBS', 'wells_type': 'All wells'}, {'step_type': 'Wait timer', 'wait_timer': 5.0}, {'step_type': 'Transfer volume to wells', 'volume_amount': '200', 'liquid_type': 'DPBS', 'wells_type': 'All wells'}, {'step_type': 'Wait timer', 'wait_timer': 5.0}, {'step_type': 'Transfer volume to wells', 'volume_amount': '200', 'liquid_type': 'DPBS', 'wells_type': 'All wells'}, {'step_type': 'Wait timer', 'wait_timer': 5.0}, {'step_type': 'Vacuum'}]

pump_step_data

### For Dotblot first try for TV-TMD-35372

In [ ]:
steps = [
         {'step_type': 'Transfer volume to wells', 'volume_amount': '200', 'liquid_type': 'DPBS'}, 
         {'step_type': 'Wait timer', 'wait_timer': 60}, {'step_type': 'Vacuum'}, 
         {'step_type': 'Transfer volume to wells', 'volume_amount': '100', 'liquid_type': 'Coating protein'}, 
         {'step_type': 'Wait timer', 'wait_timer': 480}, 
         {'step_type': 'Vacuum'}, 
         {'step_type': 'Transfer volume to wells', 'volume_amount': '200', 'liquid_type': 'Blocking buffer'}, 
         {'step_type': 'Wait timer', 'wait_timer': 480}, 
         {'step_type': 'Vacuum'}, 
         {'step_type': 'Transfer volume to wells', 'volume_amount': '100', 'liquid_type': 'Pos/Neg control'}, 
         {'step_type': 'Transfer volume to wells', 'volume_amount': '100', 'liquid_type': 'Samples'}, 
         {'step_type': 'Wait timer', 'wait_timer': 600}, 
         {'step_type': 'Vacuum'}, 
         {'step_type': 'Transfer volume to wells', 'volume_amount': '100', 'liquid_type': 'Conjugate'}, 
         {'step_type': 'Wait timer', 'wait_timer': 480}, 
         {'step_type': 'Vacuum'}, 
         {'step_type': 'Transfer volume to wells', 'volume_amount': '200', 'liquid_type': 'DPBS'}, 
         {'step_type': 'Wait timer', 'wait_timer': 60}, 
         {'step_type': 'Vacuum'},  
         {'step_type': 'Transfer volume to wells', 'volume_amount': '200', 'liquid_type': 'DPBS'}, 
         {'step_type': 'Wait timer', 'wait_timer': 60}, 
         {'step_type': 'Vacuum'},  
         {'step_type': 'Transfer volume to wells', 'volume_amount': '200', 'liquid_type': 'DPBS'}, 
         {'step_type': 'Wait timer', 'wait_timer': 60}, 
         {'step_type': 'Vacuum'}
]

steps

In [ ]:
dotblot_method.pump_steps_data = pump_step_data
dotblot_method.sample_eppendorf_positions = [3,4,5,6]

dotblot_method.generate_pump_step_instruction_files()

In [4]:
dotblot_method.transfer_volume_to_wells(dotblot_method.dpbs_lw_name, "96 Deep Well[001]", 100)

#### GENERAL DILUTION TESTS

In [182]:
reload(utils)
reload(GeneralDilution)

gen_dil = GeneralDilution.GeneralDilution()

In [183]:
sample_dilution_data = {'Dilution step': [1.0, 2.0, 3.0], 'Initial concentration': [150.0, 0.7, 0.003], 'Withdrawn volume': [4.0, 4.0, 30.0], 'Final concentration': [0.7, 0.003, 0.0001], 'Assay buffer volume': [853.1428571428572, 929.3333333333333, 869.9999999999999], 'Sample volume': [4.0, 4.0, 30.0], 'Final Volume': [857.1428571428572, 933.3333333333333, 899.9999999999999]}
gen_dil.sample_dilution_data = sample_dilution_data
gen_dil.n_sample_dilution_steps = len(gen_dil.sample_dilution_data["Assay buffer volume"])
gen_dil.n_samples = 7
gen_dil.buffer_type = "DPBS"
gen_dil.sample_lw_origin = "Falcon15"
gen_dil.sample_lw_dest = "2R Vial"
gen_dil.used_labware_pos = dict.fromkeys(gen_dil.last_labware_pos, 0)

In [184]:
gen_dil.general_dilution()

[1, 2, 3, 4, 5, 6, 7]

In [185]:
gen_dil.used_labware_pos

{'Falcon15': 7,
 'Falcon50': 0,
 'Eppendorf': 0,
 'DeepWell': 21,
 '2R Vial': 7,
 '8R Vial': 0,
 'CustomVialHolder': 0,
 'AssayBuffer': 0,
 'DPBS': 0}

In [125]:
gen_dil.next_labware_pos(utils.LabwareNames["Falcon50"])

2

### VOLUME TRANSFER

In [51]:
import VolumeTransfer
from importlib import reload
from utils import *

In [49]:
reload(VolumeTransfer)
vol_tr = VolumeTransfer.VolumeTransfer()

In [ ]:
VolumeTransfer.dilution_position_def()

In [ ]:
vol_tr.sample_lw_origin = "Eppendorf"
vol_tr.n_samples = 3
vol_tr.sample_lw_dest = "DeepWell"
vol_tr.n_sample_repetitions = 4
vol_tr.volume_transfered = 50
vol_tr.used_labware_pos = dict.fromkeys(vol_tr.used_labware_pos, 0) # reset dict

vol_tr.volume_transfer()

#### TESTS


In [34]:
step_types = []

step_types.append("timer")


step_types.append("end")

pd.DataFrame(step_types).to_csv("config - step_types.csv", header=["step_type"], index=False, lineterminator=";\n") # generate CSV file


In [57]:
val = {lw: 1 for lw in utils.LabwareNames}

val


{'Falcon15': 1,
 'Falcon50': 1,
 'Eppendorf': 1,
 'DeepWell': 1,
 '_2R_Holder': 1,
 '_8R_Holder': 1,
 'CustomVialHolder': 1,
 'AssayBuffer': 1,
 'DPBS': 1}

## Calculate labware type for each reagent based on volumes used in the method

In [39]:
reload(utils)

<module 'utils' from 'c:\\Users\\DPerez36\\OneDrive - JNJ\\Documents\\tecan_gui\\utils.py'>

In [7]:
step_types = [
                {"step_type": "Transfer volume to wells", "volume_amount": "200", "liquid_type": "DPBS"}, 
                {"step_type": "Wait timer", "wait_timer": 60},
                {"step_type": "Vacuum"}, 
                {"step_type": "Transfer volume to wells", "volume_amount": "100", "liquid_type": "Coating protein"}, 
                {"step_type": "Wait timer", "wait_timer": 480}, 
                {"step_type": "Vacuum"}, 
                {"step_type": "Transfer volume to wells", "volume_amount": "200", "liquid_type": "Blocking buffer"}, 
                {"step_type": "Wait timer", "wait_timer": 480}, 
                {"step_type": "Vacuum"}, 
                {"step_type": "Transfer volume to wells", "volume_amount": "100", "liquid_type": "Pos/Neg control"}, 
                {"step_type": "Transfer volume to wells", "volume_amount": "100", "liquid_type": "Samples"}, 
                {"step_type": "Wait timer", "wait_timer": 600}, 
                {"step_type": "Vacuum"}, 
                {"step_type": "Transfer volume to wells", "volume_amount": "100", "liquid_type": "Conjugate"}, 
                {"step_type": "Wait timer", "wait_timer": 480}, 
                {"step_type": "Vacuum"}, 
                {"step_type": "Transfer volume to wells", "volume_amount": "200", "liquid_type": "DPBS"}, 
                {"step_type": "Wait timer", "wait_timer": 60}, 
                {"step_type": "Vacuum"},  
                {"step_type": "Transfer volume to wells", "volume_amount": "200", "liquid_type": "DPBS"}, 
                {"step_type": "Wait timer", "wait_timer": 60}, 
                {"step_type": "Vacuum"},  
                {"step_type": "Transfer volume to wells", "volume_amount": "200", "liquid_type": "DPBS"}, 
                {"step_type": "Wait timer", "wait_timer": 60}, 
                {"step_type": "Vacuum"}]

REAGENTS = ["samples", "pos_ctr", "neg_ctr", "dpbs", "conjugate", "blocking_buffer", "coating_protein"]
n_samples = 4

In [ ]:
def calculate_total_volumes(step_types):
    total_volume = {}

    for step in step_types:
        if step["step_type"] == "Transfer volume to wells":
            if step["liquid_type"] not in total_volume:
                total_volume[step["liquid_type"]] = 0 # create key
            
            total_volume[step["liquid_type"]] = total_volume[step["liquid_type"]] + float(step["volume_amount"])/1000 # because volume amount is in uL and we want it in mL

    # del(total_volume["Pos/Neg control"]) # remove this key because it is not needed
    total_volume["Pos_ctr"] = total_volume.pop("Pos/Neg control") # change name of key
    total_volume["Neg_ctr"] = total_volume["Pos_ctr"] # copy the value into a new key

    # multiply by number of samples
    for key in total_volume:
        total_volume[key] *= n_samples
        total_volume[key] *= 3 # because each sample is triplicated (placed in 3 wells)
        total_volume[key] = round(total_volume[key], 2)


    pos_ctr_sample_vol = round(pos_control_dilution_data["Sample volume"].sum(), 1)
    neg_ctr_sample_vol = round(neg_control_dilution_data["Sample volume"].sum(), 1)
    buffer_vol = round(pos_control_dilution_data["Assay buffer volume"].sum(), 2) + round(neg_control_dilution_data["Assay buffer volume"].sum(), 2)
    
    total_volume["Pos_ctr"] = total_volume["Pos_ctr"] + pos_ctr_vol
    total_volume["Neg_ctr"] = total_volume["Neg_ctr"] + neg_ctr_vol
    # if buffer type is DPBS or Assay buffer, choose correct key name
    buffer_type = "DPBS"
    total_volume[buffer_type] = total_volume[buffer_type] + buffer_vol

In [15]:
total_volume = {}

for step in step_types:
    if step["step_type"] == "Transfer volume to wells":
        if step["liquid_type"] not in total_volume:
            total_volume[step["liquid_type"]] = 0 # create key
        
        total_volume[step["liquid_type"]] = total_volume[step["liquid_type"]] + float(step["volume_amount"])/1000 # because volume amount is in uL and we want it in mL

# del(total_volume["Pos/Neg control"]) # remove this key because it is not needed
total_volume["Pos_ctr"] = total_volume.pop("Pos/Neg control") # change name of key
total_volume["Neg_ctr"] = total_volume["Pos_ctr"] # copy the value into a new key

total_volume

{'DPBS': 0.8,
 'Coating protein': 0.1,
 'Blocking buffer': 0.2,
 'Samples': 0.1,
 'Conjugate': 0.1,
 'Pos_ctr': 0.1,
 'Neg_ctr': 0.1}

In [16]:
for key in total_volume:
    total_volume[key] *= n_samples
    total_volume[key] *= 3 # because each sample is triplicated (placed in 3 wells)
    total_volume[key] = round(total_volume[key], 2)

total_volume

{'DPBS': 9.6,
 'Coating protein': 1.2,
 'Blocking buffer': 2.4,
 'Samples': 1.2,
 'Conjugate': 1.2,
 'Pos_ctr': 1.2,
 'Neg_ctr': 1.2}

In [20]:
def calculate_pos_neg_ctr_volumes(pos_control_dilution_data: dict, neg_control_dilution_data: dict) -> tuple:
    pos_ctr_sample_vol = round(pos_control_dilution_data["Sample volume"].sum(), 1)
    neg_ctr_sample_vol = round(neg_control_dilution_data["Sample volume"].sum(), 1)
    buffer_vol = round(pos_control_dilution_data["Assay buffer volume"].sum(), 2) + round(neg_control_dilution_data["Assay buffer volume"].sum(), 2)

    return (pos_ctr_sample_vol, neg_ctr_sample_vol, buffer_vol)


In [21]:
pos_ctr_vol, neg_ctr_vol, buffer_vol = calculate_pos_neg_ctr_volumes(pos_control_dilution_data, neg_control_dilution_data)

In [22]:
total_volume["Pos_ctr"] = total_volume["Pos_ctr"] + pos_ctr_vol
total_volume["Neg_ctr"] = total_volume["Neg_ctr"] + neg_ctr_vol
# if buffer type is DPBS or Assay buffer, choose correct key name
buffer_type = "DPBS"
total_volume[buffer_type] = total_volume[buffer_type] + buffer_vol

total_volume

{'DPBS': 8771.74,
 'Coating protein': 1.2,
 'Blocking buffer': 2.4,
 'Samples': 1.2,
 'Conjugate': 1.2,
 'Pos_ctr': 81.2,
 'Neg_ctr': 81.2}

In [83]:
def find_best_container(volumes_dict, containers_dict):
    result = {}
    
    for item, volume_needed in volumes_dict.items():
        best_container = None
        best_sum = float('inf')
        
        for container, (vol, cap) in containers_dict.items():
            if volume_needed <= cap - vol:
                current_sum = vol + cap
                if current_sum < best_sum:
                    best_sum = current_sum
                    best_container = container
        
        if best_container:
            result[item] = best_container
        else:
            result[item] = "No suitable container"
    
    return result

In [87]:
def find_best_container(reagents_vol_dict: dict, labware_dict: dict):
    """
    Assigns the best labware type for each reagent considering the volume needed.
    """

    result = {}
    
    for item, volume_needed in reagents_vol_dict.items():
        best_container = None
        best_sum = float('inf')
        
        for labware, (dead_vol, max_vol) in labware_dict.items():
            if volume_needed <= max_vol - dead_vol:
                current_sum = max_vol - dead_vol
                if current_sum < best_sum:
                    best_sum = current_sum
                    best_container = labware
        
        if best_container:
            result[item] = best_container
        else:
            result[item] = "VOLUME TOO BIG"
    
    return result

In [97]:
total_volume["Conjugate"] = 14201

find_best_container(total_volume, LABWARE_INFO)

{'DPBS': 'Falcon15',
 'Coating protein': 'Eppendorf',
 'Blocking buffer': 'Falcon15',
 'Pos/Neg control': 'Eppendorf',
 'Samples': 'Eppendorf',
 'Conjugate': 'VOLUME TOO BIG'}

In [48]:
utils.find_best_container(0.2, utils.LABWARE_INFO)

'Eppendorf'

In [49]:
reload(utils)

<module 'utils' from 'c:\\Users\\DPerez36\\OneDrive - JNJ\\Documents\\tecan_gui\\utils.py'>